In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import os
import smtplib
import json
import base64
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from bs4 import BeautifulSoup
from random import shuffle
chromedriver = 'C:/Users/Medcomp/Anaconda3/selenium/webdriver/Chrome/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

In [3]:
url = 'https://www.google.com'
keyword_ls = ['machine learning breakthrough', 'next gold rush', 'data science trend', 'baton rouge industry',
             'startup trend', 'democratizing data', 'icd10 business', 'chatbot', 'data science project', 
             'data pipeline', 'democratizing tools']

shuffle(keyword_ls)

def google(url, keyword):
    browser = webdriver.Chrome()
    browser.get(url)
    #Search for terms and narrow search
    search_box = browser.find_element_by_xpath('//*[@id="lst-ib"]')
    search_box.send_keys(keyword + Keys.ENTER)
    time.sleep(3)
    
    search_tools = browser.find_element_by_xpath('//*[@id="hdtb-tls"]')
    search_tools.click()
    time.sleep(3)
    
    any_time_dd = browser.find_element_by_xpath('//div[@aria-label="Any time"]')
    any_time_dd.click()
    
    past_week = browser.find_element_by_xpath('//*[@id="qdr_w"]')
    past_week.click()
    
    time.sleep(3)
    
    return browser

def retrieve_urls(browser):
    
    results = browser.find_elements_by_xpath('//*[@id="rso"]/div')
    urls = []
    for result in results:
        temp = result.find_elements_by_xpath('//a')
        urls.append(temp)
    time.sleep(5)
    return browser, urls


    
def scrape(page_num, url, keyword):
    complete_urls = []
    
    goog_browser = google(url, keyword)  

    for r in range(page_num):
        time.sleep(3)
        goog_browser, temp_urls = retrieve_urls(goog_browser)
        for t, temp in enumerate(temp_urls):
            i = 0
            if t < (page_num - 2):
                for t in temp:
                    if len(t.text) > 20:
                        i += 1
                        text = str(t.text)
                        href = str(t.get_attribute('href'))
                        complete_urls.append(' ' + keyword + ': ' + str(r) + '_' + str(i))
                        complete_urls.append(text)
                        complete_urls.append(href)
                        complete_urls.append('\n')


        next_btn = goog_browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]')
        next_btn.click()
        
    message = '\n'.join(complete_urls)
    goog_browser.close()
    return message
     
            
def send_email(message, sender, receiver):
    now_string = time.strftime('%m_%d_%Y_%H:%M')
    msg = MIMEMultipart('alternative')
    msg['Subject'] = 'Google Scrape ' + now_string
    msg['From'] = sender
    msg['To'] = receiver
    body = MIMEText(message, 'plain')
    msg.attach(body)
    
    gmail_user = 'valesco19@gmail.com'
    gmail_pwd = '1QAZCDE35TGB'

    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.login(gmail_user, gmail_pwd)
    server.sendmail(sender, receiver, msg.as_string())
    server.close()
    
    file = '/home/valesco/Datasets/Google_Scraps/Google_scrap' + now_string + '.txt'
    with open(file, 'w') as f:
        f.write("\n".join(map(lambda x: str(x), message)))
        f.close()
    print('email sent ' + now_string)

#while True:
#try:
msg = []
for key in keyword_ls:
    print("Key: ", key)
    msg.append(scrape(3, url, key))

msg = '\n'.join(msg)

sender = 'valesco19@gmail.com'
receiver = 'valesco19@gmail.com'

send_email(msg, sender, receiver)
#time.sleep(86400)
#except:
    #print('email failed!')
    #time.sleep(3200)

Key:  baton rouge industry
Key:  data science trend
Key:  data pipeline
Key:  icd10 business
Key:  data science project
Key:  democratizing data
Key:  machine learning breakthrough
Key:  democratizing tools
Key:  chatbot
Key:  next gold rush
Key:  startup trend
email sent 12_05_2016_23:52
